In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path

In [3]:
path=Path('/kaggle/data_science_bowl')
path

PosixPath('/kaggle/data_science_bowl')

### Read Data

In [4]:
def read_data():
    train_df = pd.read_csv(path/'train.csv')
    test_df = pd.read_csv(path/'test.csv')
    train_labels_df = pd.read_csv(path/'train_labels.csv')
    sample_submission_df = pd.read_csv(path/'sample_submission.csv')
    return train_df, test_df, train_labels_df, sample_submission_df

In [5]:
train_df, test_df, train_labels_df, sample_submission_df = read_data()

In [6]:
train_df.head()

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
0,27253bdc,45bb1e1b6b50c07b,2019-09-06T17:53:46.937Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE
1,27253bdc,17eeb7f223665f53,2019-09-06T17:54:17.519Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK
2,77261ab5,0848ef14a8dc6892,2019-09-06T17:54:56.302Z,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",0001e90f,1,2000,0,Sandcastle Builder (Activity),Activity,MAGMAPEAK
3,b2dba42b,0848ef14a8dc6892,2019-09-06T17:54:56.387Z,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,2,3010,53,Sandcastle Builder (Activity),Activity,MAGMAPEAK
4,1bb5fbdb,0848ef14a8dc6892,2019-09-06T17:55:03.253Z,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,3,3110,6972,Sandcastle Builder (Activity),Activity,MAGMAPEAK


In [7]:
test_df.head()

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
0,27253bdc,0ea9ecc81a565215,2019-09-10T16:50:24.910Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE
1,27253bdc,c1ea43d8b8261d27,2019-09-10T16:50:55.503Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK
2,27253bdc,7ed86c6b72e725e2,2019-09-10T16:51:51.805Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Magma Peak - Level 2,Clip,MAGMAPEAK
3,27253bdc,7e516ace50e7fe67,2019-09-10T16:53:12.825Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Crystal Caves - Level 1,Clip,CRYSTALCAVES
4,7d093bf9,a022c3f60ba547e7,2019-09-10T16:54:12.115Z,"{""version"":""1.0"",""round"":0,""event_count"":1,""ga...",00abaee7,1,2000,0,Chow Time,Game,CRYSTALCAVES


In [8]:
train_labels_df.head()

,game_session,installation_id,title,num_correct,num_incorrect,accuracy,accuracy_group
0,6bdf9623adc94d89,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3
1,77b8ee947eb84b4e,0006a69f,Bird Measurer (Assessment),0,11,0.0,0
2,901acc108f55a5a1,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3
3,9501794defd84e4d,0006a69f,Mushroom Sorter (Assessment),1,1,0.5,2
4,a9ef3ecb3d1acc6a,0006a69f,Bird Measurer (Assessment),1,0,1.0,3


## Feature Engineering

In [9]:
def extract_time_features(df):
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['date'] = df['timestamp'].dt.date
    df['month'] = df['timestamp'].dt.month
    df['hour'] = df['timestamp'].dt.hour
    df['year'] = df['timestamp'].dt.year
    df['dayofweek'] = df['timestamp'].dt.dayofweek
    df['weekofyear'] = df['timestamp'].dt.weekofyear
    df['dayofyear'] = df['timestamp'].dt.dayofyear
    df['quarter'] = df['timestamp'].dt.quarter
    df['is_month_start'] = df['timestamp'].dt.is_month_start    
    
    return df
    
def get_object_columns(df, columns):
    df = df.groupby(['installation_id', columns])['event_id'].count().reset_index()
    df = df.pivot_table(index = 'installation_id', columns = [columns], values = 'event_id')
    df.columns = list(df.columns)
    df.fillna(0, inplace = True)
    return df

def get_numeric_columns(df, column):
    df = df.groupby('installation_id').agg({f'{column}': ['mean', 'sum', 'min', 'max', 'std', 'skew']})
    df[column].fillna(df[column].mean(), inplace = True)
    df.columns = [f'{column}_mean', f'{column}_sum', f'{column}_min', f'{column}_max', f'{column}_std', f'{column}_skew']
    return df

def get_numeric_columns_add(df, agg_column, column):
    df = df.groupby(['installation_id', agg_column]).agg({f'{column}': ['mean', 'sum', 'min', 'max', 'std', 'skew']}).reset_index()
    df = df.pivot_table(index = 'installation_id', columns = [agg_column], values = [col for col in df.columns if col not in ['installation_id', 'type']])
    df[column].fillna(df[column].mean(), inplace = True)
    df.columns = list(df.columns)
    return df

def perform_features_engineering(train_df, test_df, train_labels_df):
    print(f'Perform feature engineering')
    numerical_columns = ['game_time']
    categorical_columns = ['type', 'world']

    comp_train_df = pd.DataFrame({'installation_id': train_df['installation_id'].unique()})
    comp_train_df.set_index('installation_id', inplace = True)
    comp_test_df = pd.DataFrame({'installation_id': test_df['installation_id'].unique()})
    comp_test_df.set_index('installation_id', inplace = True)

    test_df = extract_time_features(test_df)
    train_df = extract_time_features(train_df)

    for i in numerical_columns:
        comp_train_df = comp_train_df.merge(get_numeric_columns(train_df, i), left_index = True, right_index = True)
        comp_test_df = comp_test_df.merge(get_numeric_columns(test_df, i), left_index = True, right_index = True)
    
    for i in categorical_columns:
        comp_train_df = comp_train_df.merge(get_object_columns(train_df, i), left_index = True, right_index = True)
        comp_test_df = comp_test_df.merge(get_object_columns(test_df, i), left_index = True, right_index = True)
    
    for i in categorical_columns:
        for j in numerical_columns:
            comp_train_df = comp_train_df.merge(get_numeric_columns_add(train_df, i, j), left_index = True, right_index = True)
            comp_test_df = comp_test_df.merge(get_numeric_columns_add(test_df, i, j), left_index = True, right_index = True)
    
    
    comp_train_df.reset_index(inplace = True)
    comp_test_df.reset_index(inplace = True)
    
    print('Our training set have {} rows and {} columns'.format(comp_train_df.shape[0], comp_train_df.shape[1]))

    # get the mode of the title
    labels_map = dict(train_labels_df.groupby('title')['accuracy_group'].agg(lambda x:x.value_counts().index[0]))
    # merge target
    labels = train_labels_df[['installation_id', 'title', 'accuracy_group']]
    # replace title with the mode
    labels['title'] = labels['title'].map(labels_map)
    # get title from the test set
    comp_test_df['title'] = test_df.groupby('installation_id').last()['title'].map(labels_map).reset_index(drop = True)
    # join train with labels
    comp_train_df = labels.merge(comp_train_df, on = 'installation_id', how = 'left')
    print('We have {} training rows'.format(comp_train_df.shape[0]))
    
    return comp_train_df, comp_test_df

In [10]:
comp_train_df, comp_test_df = perform_features_engineering(train_df, test_df, train_labels_df)

Perform feature engineering


/opt/anaconda3/envs/deeplearning/lib/python3.7/site-packages/pandas/core/generic.py:6287: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
/opt/anaconda3/envs/deeplearning/lib/python3.7/site-packages/pandas/core/generic.py:3946: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  new_axis = axis.drop(labels, errors=errors)


Our training set have 17000 rows and 63 columns


/opt/anaconda3/envs/deeplearning/lib/python3.7/site-packages/ipykernel_launcher.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


We have 17690 training rows


In [11]:
comp_train_x = comp_train_df[[i for i in comp_train_df.columns if i not in ['accuracy_group', 'installation_id', 'game_session']]]

In [12]:
comp_train_df

,installation_id,title,accuracy_group,game_time_mean,game_time_sum,game_time_min,game_time_max,game_time_std,game_time_skew,Activity,...,"(game_time, skew, NONE)","(game_time, skew, TREETOPCITY)","(game_time, std, CRYSTALCAVES)","(game_time, std, MAGMAPEAK)","(game_time, std, NONE)","(game_time, std, TREETOPCITY)","(game_time, sum, CRYSTALCAVES)","(game_time, sum, MAGMAPEAK)","(game_time, sum, NONE)","(game_time, sum, TREETOPCITY)"
0,0006a69f,3,3,82491.007366,313548319,0,1520600,159789.192183,7.504204,1771.0,...,0.0,5.704184,115847.423107,57216.164087,0.0,217683.232366,6.031041e+07,121568696.0,0.0,191979623.0
1,0006a69f,0,0,82491.007366,313548319,0,1520600,159789.192183,7.504204,1771.0,...,0.0,5.704184,115847.423107,57216.164087,0.0,217683.232366,6.031041e+07,121568696.0,0.0,191979623.0
2,0006a69f,3,3,82491.007366,313548319,0,1520600,159789.192183,7.504204,1771.0,...,0.0,5.704184,115847.423107,57216.164087,0.0,217683.232366,6.031041e+07,121568696.0,0.0,191979623.0
3,0006a69f,3,2,82491.007366,313548319,0,1520600,159789.192183,7.504204,1771.0,...,0.0,5.704184,115847.423107,57216.164087,0.0,217683.232366,6.031041e+07,121568696.0,0.0,191979623.0
4,0006a69f,0,3,82491.007366,313548319,0,1520600,159789.192183,7.504204,1771.0,...,0.0,5.704184,115847.423107,57216.164087,0.0,217683.232366,6.031041e+07,121568696.0,0.0,191979623.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17685,ffc90c32,0,3,79080.932136,158478188,0,827160,109443.413015,4.028416,479.0,...,0.0,1.250166,35463.030938,154053.818961,0.0,47262.707236,2.374887e+07,98338715.0,0.0,36390603.0
17686,ffd2871d,3,3,173255.870612,424476883,0,1204349,193862.300262,2.677175,1382.0,...,0.0,0.644670,234179.431178,171544.434637,0.0,0.000000,1.125309e+08,311945968.0,0.0,0.0
17687,ffeb0b1b,3,1,214236.207784,341278279,0,800116,214417.731703,1.202195,381.0,...,0.0,0.292296,232220.599471,89430.451610,0.0,37863.533434,2.847095e+08,51170722.0,0.0,5398047.0
17688,ffeb0b1b,3,0,214236.207784,341278279,0,800116,214417.731703,1.202195,381.0,...,0.0,0.292296,232220.599471,89430.451610,0.0,37863.533434,2.847095e+08,51170722.0,0.0,5398047.0


In [13]:
comp_train_df.columns = [c if type(c) != tuple else '_'.join(c) for c in comp_train_df.columns]

In [14]:
for c, t in zip(comp_train_df.columns, comp_train_df.dtypes):
    print(c, t)

installation_id object
title int64
accuracy_group int64
game_time_mean float64
game_time_sum int64
game_time_min int64
game_time_max int64
game_time_std float64
game_time_skew float64
Activity float64
Assessment float64
Clip float64
Game float64
CRYSTALCAVES float64
MAGMAPEAK float64
NONE float64
TREETOPCITY float64
game_time_max_Activity float64
game_time_max_Assessment float64
game_time_max_Clip float64
game_time_max_Game float64
game_time_mean_Activity float64
game_time_mean_Assessment float64
game_time_mean_Clip float64
game_time_mean_Game float64
game_time_min_Activity float64
game_time_min_Assessment float64
game_time_min_Clip float64
game_time_min_Game float64
game_time_skew_Activity float64
game_time_skew_Assessment float64
game_time_skew_Clip float64
game_time_skew_Game float64
game_time_std_Activity float64
game_time_std_Assessment float64
game_time_std_Clip float64
game_time_std_Game float64
game_time_sum_Activity float64
game_time_sum_Assessment float64
game_time_sum_Clip f

In [15]:
comp_train_x.head()

,title,game_time_mean,game_time_sum,game_time_min,game_time_max,game_time_std,game_time_skew,Activity,Assessment,Clip,...,"(game_time, skew, NONE)","(game_time, skew, TREETOPCITY)","(game_time, std, CRYSTALCAVES)","(game_time, std, MAGMAPEAK)","(game_time, std, NONE)","(game_time, std, TREETOPCITY)","(game_time, sum, CRYSTALCAVES)","(game_time, sum, MAGMAPEAK)","(game_time, sum, NONE)","(game_time, sum, TREETOPCITY)"
0,3,82491.007366,313548319,0,1520600,159789.192183,7.504204,1771.0,261.0,37.0,...,0.0,5.704184,115847.423107,57216.164087,0.0,217683.232366,6.031041e+07,121568696.0,0.0,191979623.0
1,0,82491.007366,313548319,0,1520600,159789.192183,7.504204,1771.0,261.0,37.0,...,0.0,5.704184,115847.423107,57216.164087,0.0,217683.232366,6.031041e+07,121568696.0,0.0,191979623.0
2,3,82491.007366,313548319,0,1520600,159789.192183,7.504204,1771.0,261.0,37.0,...,0.0,5.704184,115847.423107,57216.164087,0.0,217683.232366,6.031041e+07,121568696.0,0.0,191979623.0
3,3,82491.007366,313548319,0,1520600,159789.192183,7.504204,1771.0,261.0,37.0,...,0.0,5.704184,115847.423107,57216.164087,0.0,217683.232366,6.031041e+07,121568696.0,0.0,191979623.0
4,0,82491.007366,313548319,0,1520600,159789.192183,7.504204,1771.0,261.0,37.0,...,0.0,5.704184,115847.423107,57216.164087,0.0,217683.232366,6.031041e+07,121568696.0,0.0,191979623.0


In [16]:
installation_id_cnt = comp_train_df.groupby('installation_id')['installation_id'].size()
installation_id_cnt[installation_id_cnt > 1]

installation_id
0006a69f    5
0006c192    3
001d0ed0    5
002db7e3    9
003372b0    6
           ..
ff90db99    7
ff9305d7    4
ff9715db    9
ffc90c32    6
ffeb0b1b    3
Name: installation_id, Length: 2587, dtype: int64

In [17]:
comp_train_df[['installation_id', 'accuracy_group']]

,installation_id,accuracy_group
0,0006a69f,3
1,0006a69f,0
2,0006a69f,3
3,0006a69f,2
4,0006a69f,3
...,...,...
17685,ffc90c32,3
17686,ffd2871d,3
17687,ffeb0b1b,1
17688,ffeb0b1b,0


## Normalize

In [18]:
"Normalize the continuous variables."
def normalize(df: pd.DataFrame, cont_names):
    "Compute the means and stds of `self.cont_names` columns to normalize them."
    means, stds = {},{}
    for n in cont_names:
        means[n], stds[n] = df[n].mean(), df[n].std()
        df[n] = (df[n]-means[n]) / (1e-7 + stds[n])

In [19]:
normalize(comp_train_df, [c for c in comp_train_df.columns if c not in ['installation_id', 'title', 'accuracy_group']])

In [20]:
comp_train_df

,installation_id,title,accuracy_group,game_time_mean,game_time_sum,game_time_min,game_time_max,game_time_std,game_time_skew,Activity,...,game_time_skew_NONE,game_time_skew_TREETOPCITY,game_time_std_CRYSTALCAVES,game_time_std_MAGMAPEAK,game_time_std_NONE,game_time_std_TREETOPCITY,game_time_sum_CRYSTALCAVES,game_time_sum_MAGMAPEAK,game_time_sum_NONE,game_time_sum_TREETOPCITY
0,0006a69f,3,3,-0.202444,-0.329926,0.0,-0.108068,-0.111270,0.649940,-0.013334,...,0.0,0.908488,-0.084377,-0.253894,0.0,0.028054,-0.325484,-0.324082,0.0,0.007949
1,0006a69f,0,0,-0.202444,-0.329926,0.0,-0.108068,-0.111270,0.649940,-0.013334,...,0.0,0.908488,-0.084377,-0.253894,0.0,0.028054,-0.325484,-0.324082,0.0,0.007949
2,0006a69f,3,3,-0.202444,-0.329926,0.0,-0.108068,-0.111270,0.649940,-0.013334,...,0.0,0.908488,-0.084377,-0.253894,0.0,0.028054,-0.325484,-0.324082,0.0,0.007949
3,0006a69f,3,2,-0.202444,-0.329926,0.0,-0.108068,-0.111270,0.649940,-0.013334,...,0.0,0.908488,-0.084377,-0.253894,0.0,0.028054,-0.325484,-0.324082,0.0,0.007949
4,0006a69f,0,3,-0.202444,-0.329926,0.0,-0.108068,-0.111270,0.649940,-0.013334,...,0.0,0.908488,-0.084377,-0.253894,0.0,0.028054,-0.325484,-0.324082,0.0,0.007949
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17685,ffc90c32,0,3,-0.217464,-0.435210,0.0,-0.155625,-0.159852,0.056372,-0.597893,...,0.0,-0.221706,-0.242691,-0.109640,0.0,-0.096095,-0.402603,-0.350554,0.0,-0.323097
17686,ffd2871d,3,3,0.197345,-0.254611,0.0,-0.129757,-0.078391,-0.174383,-0.189335,...,0.0,-0.375349,0.148674,-0.083585,0.0,-0.130525,-0.215335,-0.107131,0.0,-0.400525
17687,ffeb0b1b,3,1,0.377849,-0.311098,0.0,-0.157480,-0.058556,-0.426268,-0.642233,...,0.0,-0.464762,0.144817,-0.205906,0.0,-0.102942,0.147841,-0.404306,0.0,-0.389040
17688,ffeb0b1b,3,0,0.377849,-0.311098,0.0,-0.157480,-0.058556,-0.426268,-0.642233,...,0.0,-0.464762,0.144817,-0.205906,0.0,-0.102942,0.147841,-0.404306,0.0,-0.389040


## Data Set and Data Loader

In [21]:
from torch.utils.data import DataLoader, Dataset, SequentialSampler, RandomSampler
import torch
from torch import nn

def convert_df_to_tensor(df):
    return torch.tensor(df.values)

class BowlDataSet(Dataset):
    def __init__(self, df_x: pd.DataFrame, df_y: pd.DataFrame):
        self.x = convert_df_to_tensor(df_x).float()
        self.y = convert_df_to_tensor(df_y).long()
        if torch.cuda.is_available():
            self.x = self.x.cuda()
            self.y = self.y.cuda()
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, i):
        return self.x[i], self.y[i]
    

In [22]:
m_count = comp_train_df.shape[0]
pct = 0.9
x_size = int(m_count * pct)
comp_train_y = comp_train_df['accuracy_group']
train_ds = BowlDataSet(comp_train_x[:x_size], comp_train_y[:x_size])
valid_ds = BowlDataSet(comp_train_x[x_size:], comp_train_y[x_size:])

In [23]:
len(train_ds), len(valid_ds), x_size

(15921, 1769, 15921)

In [24]:
assert (len(train_ds) + len(valid_ds) == len(comp_train_df))

In [25]:
bs = 64

In [26]:
train_dl = DataLoader(train_ds, bs, shuffle=True, drop_last=True)
valid_dl = DataLoader(valid_ds, bs, shuffle=False)

In [27]:
batch_sample = next(iter(train_dl))

In [28]:
batch_sample[0].shape, batch_sample[1].shape

(torch.Size([64, 63]), torch.Size([64]))

## Model

In [29]:
m = comp_train_x.shape[1]
m

63

In [30]:
number_hidden = 400
number_hidden_2 = 200
number_hidden_3 = 100

In [31]:
categories = len(comp_train_df['accuracy_group'].unique())
categories

4

In [32]:
def combinedLayer(m_in, m_out):
    return (nn.Linear(m_in, m_out), nn.ReLU(inplace=True), nn.BatchNorm1d(m_out, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True))

model = nn.Sequential(*combinedLayer(m, number_hidden), *combinedLayer(number_hidden, number_hidden_2),
                      *combinedLayer(number_hidden_2, number_hidden_3),
                      nn.Linear(number_hidden_3, categories))
model

Sequential(
  (0): Linear(in_features=63, out_features=400, bias=True)
  (1): ReLU(inplace=True)
  (2): BatchNorm1d(400, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): Linear(in_features=400, out_features=200, bias=True)
  (4): ReLU(inplace=True)
  (5): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): Linear(in_features=200, out_features=100, bias=True)
  (7): ReLU(inplace=True)
  (8): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (9): Linear(in_features=100, out_features=4, bias=True)
)

In [33]:
for l in model:
    if type(l) == nn.Linear:
        nn.init.kaiming_uniform_(l.weight)

## Loss and Optimizers

In [34]:
import torch.nn.functional as F
loss_func = F.cross_entropy
# loss_func = F.mse_loss

In [35]:
lr = 1e-4
lr

0.0001

In [36]:
import torch.optim as optim
optimizer = optim.Adam(model.parameters(), lr, [0.9, 0.99])
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: [0.9, 0.99]
    eps: 1e-08
    lr: 0.0001
    weight_decay: 0
)

## Accuracy functions

In [37]:
# quadratic weighted kappa
def qwk3(a1, a2, max_rat=3):
    '''
    a1 - ground truth
    a2 - predicted values
    '''
    assert(len(a1) == len(a2))
    a1 = np.asarray(a1, dtype=int)
    a2 = np.asarray(a2, dtype=int)
    hist1 = np.zeros((max_rat + 1, ))
    hist2 = np.zeros((max_rat + 1, ))
    o = 0
    for k in range(a1.shape[0]):
        i, j = a1[k], a2[k]
        hist1[i] += 1
        hist2[j] += 1
        o +=  (i - j) * (i - j)
    e = 0
    for i in range(max_rat + 1):
        for j in range(max_rat + 1):
            e += hist1[i] * hist2[j] * (i - j) * (i - j)
    e = e / a1.shape[0]
    return 1 - o / e

In [38]:
def mse(output, targ): 
    return (output.float() - targ.float()).pow(2).mean()

In [39]:
def accuracy(out, yb): 
    return (torch.argmax(out, dim=1)==yb).float().mean()

In [40]:
mse_text_output = torch.randn(32, 4).float()
targ = torch.randn(32).long()
mse_text_output_argmax = mse_text_output.argmax(-1)
mse(mse_text_output_argmax, targ)

tensor(3.0312)

In [41]:
qwk3(mse_text_output_argmax, targ)

-0.20496894409937894

In [42]:
accuracy(mse_text_output, targ)

tensor(0.2188)

## Training Loop

In [43]:
#export
class Learner():
    def __init__(self, model, opt): 
        self.max_qwk = 0
        self.max_acc = 0
        self.epoch = 0
        self.tot_train_loss = 0.0
        self.model = model
        self.opt = opt
        if torch.cuda.is_available():
            self.model = self.model.cuda()
    
    def __str__(self):
        return f'* max quadratic weighted kappa: {self.max_qwk}, max acc: {self.max_acc}'

def eval_model(learner: Learner, loss_func, train_dl, valid_dl):
    learner.model.eval()
    with torch.no_grad():
        tot_loss, tot_mse, tot_acc, tot_acc_qwk = 0., 0., 0., 0.
        for xb, yb in valid_dl:
            pred = learner.model(xb)
            tot_loss += loss_func(pred, yb)
            tot_acc += accuracy(pred, yb)
            tot_acc_qwk += qwk3(yb.cpu(), pred.argmax(-1).cpu())
        nv = len(valid_dl)
        qwk = tot_acc_qwk / nv
        if qwk > learner.max_qwk:
            learner.max_qwk = qwk
            print(learner)
        acc = tot_acc / nv
        if acc > learner.max_acc:
            learner.max_acc = acc
            print(learner)
        print(f'{learner.epoch} trainloss: {learner.tot_train_loss / len(train_dl)} valid loss: {tot_loss / nv} qwk: {qwk} acc: {acc}')

def fit(epochs: int, loss_func, train_dl, valid_dl, learner: Learner):
    for epoch in range(epochs):
        learner.model.train()
        learner.tot_train_loss = 0.
        for xb, yb in train_dl:
            loss = loss_func(learner.model(xb), yb)
            learner.tot_train_loss += loss
            loss.backward()
            learner.opt.step()
            learner.opt.zero_grad()
        
        learner.epoch = epoch
        eval_model(learner, loss_func, train_dl, valid_dl)

In [44]:
learner = Learner(model, optimizer)
fit(5000, loss_func, train_dl, valid_dl, learner)

* max quadratic weighted kappa: 0.056062574527335464, max acc: 0
* max quadratic weighted kappa: 0.056062574527335464, max acc: 0.4624210596084595
0 trainloss: 1.460706353187561 valid loss: 1.3553727865219116 qwk: 0.056062574527335464 acc: 0.4624210596084595
1 trainloss: 1.3138335943222046 valid loss: 1.3476417064666748 qwk: 0.014555855016345043 acc: 0.45537078380584717
2 trainloss: 1.2585078477859497 valid loss: 1.3735283613204956 qwk: 0.01919298204996982 acc: 0.4512195587158203
* max quadratic weighted kappa: 0.056062574527335464, max acc: 0.5083841681480408
3 trainloss: 1.2343116998672485 valid loss: 1.317657709121704 qwk: 0.0020716301806275356 acc: 0.5083841681480408
4 trainloss: 1.226149320602417 valid loss: 1.2525763511657715 qwk: -0.0055927585459097495 acc: 0.5067100524902344
5 trainloss: 1.2148714065551758 valid loss: 1.224425196647644 qwk: -0.010156727373331783 acc: 0.5050359964370728
6 trainloss: 1.206978678703308 valid loss: 1.2279057502746582 qwk: -0.016809090118283437 acc:

69 trainloss: 1.1651986837387085 valid loss: 1.2791842222213745 qwk: 0.012265997398714253 acc: 0.5132023096084595
70 trainloss: 1.1626003980636597 valid loss: 1.2497204542160034 qwk: 0.023745918432956616 acc: 0.5206609964370728
71 trainloss: 1.1618626117706299 valid loss: 1.2146196365356445 qwk: 0.021478707061010422 acc: 0.5171085596084595
72 trainloss: 1.1605033874511719 valid loss: 1.2459686994552612 qwk: 0.005396020382478749 acc: 0.5098540782928467
73 trainloss: 1.1604734659194946 valid loss: 1.272810697555542 qwk: 0.00876985198209663 acc: 0.4663273096084595
74 trainloss: 1.1615595817565918 valid loss: 1.226657509803772 qwk: 0.012032488439827191 acc: 0.5137603282928467
75 trainloss: 1.1645694971084595 valid loss: 1.200903296470642 qwk: 0.007254672606991871 acc: 0.5076219439506531
76 trainloss: 1.1630967855453491 valid loss: 1.2185903787612915 qwk: -0.01330437444664179 acc: 0.49701929092407227
77 trainloss: 1.1598412990570068 valid loss: 1.2621487379074097 qwk: -0.0012706734442351011

142 trainloss: 1.1487737894058228 valid loss: 1.225595474243164 qwk: 0.04246786620937626 acc: 0.5207698345184326
143 trainloss: 1.147386908531189 valid loss: 1.1987452507019043 qwk: 0.03364634667790428 acc: 0.517979621887207
144 trainloss: 1.147418737411499 valid loss: 1.2128026485443115 qwk: 0.03273936860891678 acc: 0.5065059065818787
145 trainloss: 1.1499754190444946 valid loss: 1.2366464138031006 qwk: 0.023656566044459888 acc: 0.5157474875450134
146 trainloss: 1.1490693092346191 valid loss: 1.2368658781051636 qwk: 0.011759716822031077 acc: 0.465524286031723
147 trainloss: 1.1479771137237549 valid loss: 1.2602373361587524 qwk: 0.0210084468393334 acc: 0.5154344439506531
148 trainloss: 1.1508973836898804 valid loss: 1.3158918619155884 qwk: 0.028326024881009455 acc: 0.5165505409240723
149 trainloss: 1.1519190073013306 valid loss: 1.211836576461792 qwk: 0.0057511224932510485 acc: 0.5034707188606262
150 trainloss: 1.1477866172790527 valid loss: 1.2556259632110596 qwk: -0.00052338860784432

213 trainloss: 1.137945294380188 valid loss: 1.2485440969467163 qwk: -0.011572298522080585 acc: 0.4947871267795563
214 trainloss: 1.1398072242736816 valid loss: 1.229112982749939 qwk: 0.0075743487241845065 acc: 0.5042737722396851
215 trainloss: 1.1406582593917847 valid loss: 1.2011127471923828 qwk: -0.00909359718314952 acc: 0.5042737722396851
216 trainloss: 1.14109206199646 valid loss: 1.2297718524932861 qwk: -0.007460720811415518 acc: 0.49701929092407227
217 trainloss: 1.1381759643554688 valid loss: 1.206721544265747 qwk: -0.004474260653278709 acc: 0.4933580160140991
218 trainloss: 1.139210820198059 valid loss: 1.2234748601913452 qwk: 0.0035516745577144665 acc: 0.49422910809516907
219 trainloss: 1.1429095268249512 valid loss: 1.204516887664795 qwk: 0.03447985340781255 acc: 0.5159925222396851
220 trainloss: 1.137834072113037 valid loss: 1.2286394834518433 qwk: 0.0011552784420034554 acc: 0.4975773096084595
221 trainloss: 1.1391208171844482 valid loss: 1.2397593259811401 qwk: -0.00161305

284 trainloss: 1.1321967840194702 valid loss: 1.2193267345428467 qwk: 0.010631143789799267 acc: 0.510861337184906
285 trainloss: 1.1323330402374268 valid loss: 1.2415401935577393 qwk: 0.0056483746905732135 acc: 0.5053898096084595
286 trainloss: 1.1307319402694702 valid loss: 1.2674674987792969 qwk: 0.0027404912972886575 acc: 0.4989383816719055
287 trainloss: 1.1301422119140625 valid loss: 1.2140753269195557 qwk: -0.013231020019171171 acc: 0.4888937473297119
288 trainloss: 1.1318464279174805 valid loss: 1.207777976989746 qwk: 0.008422340662887409 acc: 0.5003675222396851
289 trainloss: 1.1308157444000244 valid loss: 1.2355308532714844 qwk: 0.018869115260628733 acc: 0.5042737722396851
290 trainloss: 1.1340150833129883 valid loss: 1.2150962352752686 qwk: 0.013086092980971917 acc: 0.5048317909240723
291 trainloss: 1.126832365989685 valid loss: 1.2397716045379639 qwk: 0.005640058624332934 acc: 0.49646124243736267
292 trainloss: 1.130137324333191 valid loss: 1.2373157739639282 qwk: 0.03264580

...

2863 trainloss: 1.033552885055542 valid loss: 1.284029245376587 qwk: -0.00564627392451557 acc: 0.5003675222396851
2864 trainloss: 1.0382801294326782 valid loss: 1823.22412109375 qwk: 0.009766539937297742 acc: 0.49701929092407227
2865 trainloss: 1.0365681648254395 valid loss: 1.2781742811203003 qwk: 0.021442639532429805 acc: 0.5081800222396851
2866 trainloss: 1.0347384214401245 valid loss: 2846.117919921875 qwk: 0.013354339318685318 acc: 0.48083624243736267
2867 trainloss: 1.0355771780014038 valid loss: 1.2990211248397827 qwk: -0.0037959844109525798 acc: 0.5037156939506531
2868 trainloss: 1.036612629890442 valid loss: 1.2809982299804688 qwk: -0.003376464739932452 acc: 0.5031576752662659
2869 trainloss: 1.0356113910675049 valid loss: 3306.251708984375 qwk: 0.01670253753331195 acc: 0.5132023096084595
2870 trainloss: 1.0395026206970215 valid loss: 4538.3134765625 qwk: -0.00044580949227471245 acc: 0.50950026512146
2871 trainloss: 1.0395501852035522 valid loss: 1.2245908975601196 qwk: 0.0098

2936 trainloss: 1.0378385782241821 valid loss: 17116.884765625 qwk: 0.01758171411122669 acc: 0.5167547464370728
2937 trainloss: 1.0336940288543701 valid loss: 14159.947265625 qwk: 0.02429735181487732 acc: 0.5137603282928467
2938 trainloss: 1.0370229482650757 valid loss: 22112.041015625 qwk: 0.010806089186650745 acc: 0.5070639252662659
2939 trainloss: 1.034466028213501 valid loss: 22475.107421875 qwk: 0.01452390462606821 acc: 0.5137603282928467
2940 trainloss: 1.0362955331802368 valid loss: 18671.921875 qwk: 0.004571235100270814 acc: 0.5109701752662659
2941 trainloss: 1.0361511707305908 valid loss: 9837.427734375 qwk: -0.000909841016158462 acc: 0.5043418407440186
2942 trainloss: 1.0356687307357788 valid loss: 11732.048828125 qwk: 0.014902135275702388 acc: 0.5059478282928467
2943 trainloss: 1.0339123010635376 valid loss: 2951.82373046875 qwk: 0.03814037024245561 acc: 0.5179116129875183
2944 trainloss: 1.0353083610534668 valid loss: 20047.884765625 qwk: -0.005451739453771083 acc: 0.505035

3008 trainloss: 1.033202886581421 valid loss: 13347.5478515625 qwk: 0.04807591913591354 acc: 0.49813535809516907
3009 trainloss: 1.0343502759933472 valid loss: 14161.53125 qwk: 0.011659544468657222 acc: 0.4791621267795563
3010 trainloss: 1.0321614742279053 valid loss: 3624.263916015625 qwk: 0.029598249693946643 acc: 0.4802781939506531
3011 trainloss: 1.037420630455017 valid loss: 2258.41162109375 qwk: 0.025413530312775403 acc: 0.5120862722396851
3012 trainloss: 1.0358302593231201 valid loss: 6162.9755859375 qwk: 0.05181014662585824 acc: 0.5251660346984863
3013 trainloss: 1.0366748571395874 valid loss: 1673.088623046875 qwk: 0.01972187894335239 acc: 0.5159925222396851
3014 trainloss: 1.0315933227539062 valid loss: 10369.384765625 qwk: 0.04253707091900526 acc: 0.5221309065818787
3015 trainloss: 1.0345064401626587 valid loss: 8955.703125 qwk: 0.030256402670546217 acc: 0.5234511494636536
3016 trainloss: 1.0372341871261597 valid loss: 13389.8232421875 qwk: 0.026886681153975002 acc: 0.516550

3080 trainloss: 1.0306223630905151 valid loss: 14437.4580078125 qwk: 0.003991963885714466 acc: 0.5109701752662659
3081 trainloss: 1.0319141149520874 valid loss: 25171.349609375 qwk: -0.02143052240236426 acc: 0.48641660809516907
3082 trainloss: 1.0299423933029175 valid loss: 15895.7265625 qwk: -0.008144439876310148 acc: 0.5081800222396851
3083 trainloss: 1.032617449760437 valid loss: 13769.837890625 qwk: 0.008556052559121091 acc: 0.49164310097694397
3084 trainloss: 1.0348039865493774 valid loss: 22564.923828125 qwk: 0.010716921199451692 acc: 0.5070639252662659
3085 trainloss: 1.0306168794631958 valid loss: 14738.46875 qwk: 0.00864501116537482 acc: 0.5031576752662659
3086 trainloss: 1.0331463813781738 valid loss: 12026.50390625 qwk: 0.020390300038675497 acc: 0.5087380409240723
3087 trainloss: 1.0292679071426392 valid loss: 6396.68505859375 qwk: -0.0048350000068096265 acc: 0.5087380409240723
3088 trainloss: 1.034030556678772 valid loss: 5602.57958984375 qwk: 0.015641959731146522 acc: 0.50

KeyboardInterrupt: 

In [ ]:
learner.max_qwk